# Group A Programming Assignment 4 Report

**Authors:** Diego Green, Jason Gutierrez, Nelson Nin, Abel Martinez 

**Course:** ICOM5015-001D Artificial Intelligence

## Task Division

| Task            | Group Member      |
|:-----------------:|:-------------------:|
| Programming     | Diego Green       |
| Debugging       | Jason Gutierrez     |
| Report Writing  | Diego Green   |
| Report Editing  | Jason Gutierrez    |
| Video Scripting | Nelson Nin   |
| Video Editing   | Abel Martinez     |

# Abstract

This report details the implementation of an artificial intelligence (AI) player for the game of Domino, with a combination of the Minimax algorithm and Monte Carlo simulations (MCTS). Minimax is traditionally effective for deterministic games, but given the inherent uncertainty in Domino primarily due to hidden information about opponents' tiles integrating Monte Carlo simulation significantly enhances decision-making under uncertainty. This combination enables our AI agent to approximate the outcomes of possible moves more accurately, outperforming traditional deterministic Minimax. Experimental testing demonstrates that our enhanced approach not only improves strategic decisions but also effectively handles the uncertainty inherent in a Domino match, making our AI competitive against human opponents.

# Introduccion 


Artificial intelligence has extensively explored adversarial board games such as chess, Go, and dominoes, which provide valuable platforms for developing sophisticated decision-making algorithms. Traditional adversarial search algorithms like Minimax perform well in fully observable scenarios, but their performance diminishes in games with hidden information, such as Domino. To address this limitation, we combine Minimax with Monte Carlo simulations, which allows our AI player to consider stochastic elements and hidden states effectively. Monte Carlo simulations involve repeatedly sampling uncertain elements in this case, unknown domino tiles and performing simulated play-outs to statistically estimate optimal moves. This project demonstrates how integrating stochastic sampling into traditional Minimax significantly improves gameplay decisions, justifying our method selection over purely deterministic or overly complex solutions.

We chose Monte Carlo Simulation combined with Minimax as our primary adversarial search technique due to the partially observable nature of Domino. 

Here's why:

- Stochastic Games: Effective but potentially overly complex for Domino’s manageable state space.
- Partially Observable Games: Naturally fits Domino, but without explicit probabilistic modeling might become inefficient.
- Monte Carlo Simulation (Selected Method): Offers a balance of complexity and effectiveness, addressing uncertainty via stochastic sampling, making it ideal for Domino.
- Averaging Over Clairvoyance: Effective but similar to Monte Carlo; we preferred MCTS due to its clearer statistical foundations.
- Deep Neural Networks: Powerful but complex, requiring significant computational resources and training datasets not readily available or needed for the simplicity of Domino.

Thus, Monte Carlo-enhanced Minimax provided the optimal trade-off between simplicity and performance, explicitly addressing uncertainty in opponents' hidden tiles.

In [ ]:
import copy
import random

class Domino:

    def __init__ (self, left_side, right_side):
        self.left_side = left_side
        self.right_side = right_side
        self.value = left_side + right_side
        
    def __str__(self):
        return f"[{self.left_side}|{self.right_side}]"
    
    def __rpr__(self):
        return f"[{self.left_side}|{self.right_side}]"
        
    def is_double(self):
        return self.left_side == self.right_side
    
    def linkable(self, domino):
        if self.left_side == domino.left_side:
            return 'LL'
        if self.right_side == domino.left_side:
            return 'RL'
        if self.left_side == domino.right_side:
            return 'LR'
        if self.right_side == domino.right_side:
            return 'RR'
        return 'Non-compatible'
    
    def flip(self):
        temp = self.left_side
        self.left_side = self.right_side
        self.right_side = temp
        
    def points(self):
        return self.right_side + self.left_side
    
    def equals(self, domino):
        if self.left_side == domino.left_side and self.right_side == domino.right_side:
            return True
        return False
    
    def get_right(self):
        return self.right_side
    
    def get_left(self):
        return self.left_side
    
class GameState:
    
    def __init__(self, domino):
        self.first_domino = domino
        self.left_edge = domino.get_left()
        self.right_edge = domino.get_right()
        self.placed_dominoes = set()
        self.left_side = []
        self.right_side = []
        self.player_skipped_edges = set()
        self.consecutive_skips = 0
        # self.current_player = 0
        self.terminal = False
        
    def place_domino(self, domino, player):
        # Out of bounds
        if domino.get_right() == -1 and domino.get_left() == -1:
            return -2
        
        # Empty hand
        if len(player.hand) == 0:
            self.terminal = True
        
        # Chose wrongly
        if domino.get_right() == -3 and domino.get_left() == -3:
            return 1
        
        # Has to skip               
        if domino.get_right() == -4 and domino.get_left() == -4:
            self.consecutive_skips += 1
            if self.consecutive_skips == 2:
                self.terminal = True
            return -1
        
        compatibility = domino.linkable(Domino(self.left_edge, self.right_edge))
        if compatibility != 'Non-compatible':
            self.placed_dominoes.add(domino)            
            if compatibility == 'LR':
                self.right_edge = domino.get_right()
                self.right_side.append(domino)
            elif compatibility == 'LL':
                self.left_edge = domino.get_right()
                self.left_side.append(domino)
            elif compatibility == 'RL':
                self.left_edge = domino.get_left()
                self.left_side.append(domino)
            elif compatibility == 'RR':
                self.right_edge = domino.get_left()
                self.right_side.append(domino)
            self.consecutive_skips = 0
            return 0
                
    def dominos_placed(self):
        return self.placed_dominoes.len()
    
    def print_board(self):
        cur_board = []
        cur_board.append(self.first_domino)
        for domino in self.left_side:
            if cur_board[0].linkable(domino) == 'LL':
                domino.flip()
            cur_board.insert(0, domino)
            
        for domino in self.right_side:
            if cur_board[len(cur_board)-1].linkable(domino) == 'RR':
                domino.flip()
            cur_board.append(domino)
            
        for domino in cur_board:
            print(domino, end = " ")
            
        print("")        
        
class Player:
    
    def __init__(self, name, hand):
        self.name = name
        self.hand = hand
        self.points = 0
        self.highest_initial_domino = hand[0]
        self.highest_initial_domino_index = 0
        
        index = 0
        for domino in hand:
            self.points += domino.points()
            if domino.points() > self.highest_initial_domino.points():
                self.highest_initial_domino_index = index
                self.highest_initial_domino = domino
            index+=1
            
    def place(self, choice, left_edge, right_edge):
        
        if choice < 1 or choice > len(self.hand):
            return Domino(-1,-1)
        
        # if len(self.hand) == 0:
        #     return Domino(-2,-2)
        
        if self.hand[choice-1].linkable(Domino(left_edge, right_edge)) != 'Non-compatible':
            self.points -= self.hand[choice-1].points()  
            return self.hand.pop(choice-1)
        
        for domino in self.hand:
            if domino.linkable(Domino(left_edge, right_edge)) != 'Non-compatible':
                return Domino(-3,-3)
            
        return Domino(-4,-4)

    def hand_points(self):
        points = 0
        
        for domino in self.hand:
            points += domino.points()
            
        return points
    
    def equals(self, player):
        if self.name == player.name and self.highest_initial_domino.equals(player.highest_initial_domino):
            return True
        return False
    
    def print_hand(self):
        for domino in self.hand:
            print(domino, end = " ")
            
        print("")

class AIPlayer(Player):
    def __init__(self, name, hand, simulations=50):
        super().__init__(name, hand)
        self.simulations = simulations  # Number of Monte Carlo simulations

    def place(self, game_state):
        best_move, best_score = None, float('-inf')
        for idx, domino in enumerate(self.hand):
            total_score = 0
            valid_simulations = 0
            
            for _ in range(self.simulations):
                cloned_state = self.clone_game_state(game_state)
                cloned_player = copy.deepcopy(self)

                # Try to place domino
                result = cloned_state.place_domino(domino, cloned_player)
                if result == 0:  # Valid move
                    valid_simulations += 1
                    score = self.simulate_random_game(cloned_state, cloned_player, depth=3)
                    total_score += score

            if valid_simulations > 0:
                average_score = total_score / valid_simulations
                if average_score > best_score:
                    best_score = average_score
                    best_move = idx

        return self.hand.pop(best_move) if best_move is not None else Domino(-4, -4)  # Skip if no valid moves

    def simulate_random_game(self, game_state, player, depth):
        if depth == 0 or game_state.terminal:
            return self.evaluate(game_state)

        possible_moves = [d for d in player.hand if d.linkable(Domino(game_state.left_edge, game_state.right_edge)) != 'Non-compatible']
        if not possible_moves:
            return self.evaluate(game_state)  # No valid moves, evaluate current state

        domino = random.choice(possible_moves)
        game_state.place_domino(domino, player)
        player.hand.remove(domino)

        return self.simulate_random_game(game_state, player, depth - 1)

    def evaluate(self, game_state):
        return -self.hand_points()  # Fewer points in hand is better

    def clone_game_state(self, game_state):
        cloned_game_state = GameState(Domino(game_state.first_domino.left_side, game_state.first_domino.right_side))
        cloned_game_state.left_edge = game_state.left_edge
        cloned_game_state.right_edge = game_state.right_edge
        cloned_game_state.placed_dominoes = game_state.placed_dominoes.copy()
        cloned_game_state.left_side = game_state.left_side.copy()
        cloned_game_state.right_side = game_state.right_side.copy()
        cloned_game_state.player_skipped_edges = game_state.player_skipped_edges.copy()
        cloned_game_state.consecutive_skips = game_state.consecutive_skips
        cloned_game_state.terminal = game_state.terminal
        return cloned_game_state


class Game:
    
    def __init__(self):
        self.domino_pool = []
        
    def generate_dominoes(self):
        limit = 6
        
        while  limit > 0:
            for i in range(0,limit):
                self.domino_pool.append(Domino(limit, i))
            limit -= 1
    
    def shuffle(self):
        random.shuffle(self.domino_pool)
        
    def give_dominoes(self):
        
        hand = []
        
        for index in range(7):
            hand.append(self.domino_pool.pop(index))
            
        return hand
    
game = Game()
game.generate_dominoes()
game.shuffle()

player1 = AIPlayer("DominoMaster (BOT)", game.give_dominoes())
player2 = Player("User", game.give_dominoes())

current_player = None
highest_double = 0
index = 0
domino_index = 0

for domino in player1.hand:
    index += 1
    if domino.is_double() and domino.points() > highest_double:
        current_player = player1
        domino_index = index - 1
index = 0
for domino in player2.hand:
    index += 1
    if domino.is_double() and domino.points() > highest_double:
        current_player = player2
        domino_index = index - 1
        
if current_player is None:
    if player1.highest_initial_domino.points() > player2.highest_initial_domino.points():
        current_player = player1
        domino_index = player1.highest_initial_domino_index
    else: 
        current_player = player2
        domino_index = player2.highest_initial_domino_index
    
game_state = GameState(current_player.hand.pop(domino_index))

if current_player.equals(player1):
    current_player = player2
else:
    current_player = player1
        
while not game_state.terminal:
    if current_player.equals(player2):
        print("\nCurrent Board:")
        game_state.print_board()
        print(f"\n{current_player.name}'s turn. Your hand:")
        current_player.print_hand()
        
    flag = -1
    while flag != 0 and not game_state.terminal and current_player.equals(player2):
        user_input = input("Enter the number of the domino to place (1 to " + str(len(current_player.hand)) + "). If you have no options, place a random domino to skip: ")
        
        while not user_input.isdigit() or int(user_input) < 1 or int(user_input) > len(current_player.hand):
            user_input = input("Invalid entry. Enter the number of the domino to place (1 to " + str(len(current_player.hand)) + "). If you have no options, place a random domino to skip: ")
        
        choice = int(user_input)
        
        flag = game_state.place_domino(current_player.place(choice, game_state.left_edge, game_state.right_edge), current_player)
        if flag == 1:
            print("\nSelected domino is not valid. Choose again.")
        elif flag == -1:
            print("\n" + current_player.name + " has skipped this turn.")
            flag = 0
        elif flag == -2:
            print("\nOut of index. Pick between 1 and " + str(len(current_player.hand)))

    if current_player.equals(player1):
        print(f"\n{current_player.name}'s turn.")
        placed_domino = current_player.place(game_state)
        
        if placed_domino.get_left() == -4:
            print(f"{current_player.name} had no valid dominoes and skipped the turn.")
        else:
            print(f"{current_player.name} placed domino: {placed_domino}")

        game_state.place_domino(placed_domino, current_player)
        print(f"\n{current_player.name}'s turn has ended.")
        
    current_player = player1 if current_player.equals(player2) else player2

# End Game State
print("\nGame over!")
print(player1.name + " score: " + str(player1.hand_points()))
print(player2.name + " score: " + str(player2.hand_points()))

if player1.hand_points() == player2.hand_points():
    print("The score is tied.")
elif player1.hand_points() < player2.hand_points():
    print("The Bot wins! be better...")
else:
    print("You win! GG")


Current Board:
[6|4] 

User's turn. Your hand:
[5|1] [5|3] [3|1] [2|1] [4|1] [6|0] [5|2] 

Bot Joey's turn.

Bot Joey's turn has ended.

Current Board:
[1|6] [6|4] [4|1] 

User's turn. Your hand:
[5|1] [5|3] [3|1] [2|1] [6|0] [5|2] 

Bot Joey's turn.

Bot Joey's turn has ended.

Current Board:
[3|2] [2|1] [1|6] [6|4] [4|1] 

User's turn. Your hand:
[5|1] [5|3] [3|1] [6|0] [5|2] 

Bot Joey's turn.

Bot Joey's turn has ended.

Current Board:
[5|3] [3|2] [2|1] [1|6] [6|4] [4|1] [1|0] 

User's turn. Your hand:
[5|1] [3|1] [6|0] [5|2] 

Out of index. Pick between 1 and 4

Bot Joey's turn.

Bot Joey's turn has ended.

Current Board:
[1|5] [5|3] [3|2] [2|1] [1|6] [6|4] [4|1] [1|0] [0|3] 

User's turn. Your hand:
[3|1] [6|0] [5|2] 

Bot Joey's turn.

Bot Joey's turn has ended.

Current Board:
[3|1] [1|5] [5|3] [3|2] [2|1] [1|6] [6|4] [4|1] [1|0] [0|3] 

User's turn. Your hand:
[6|0] [5|2] 

User has skipped this turn
Game over!
Bot Joey score: 13
User score: 13
The score is tied.


# Complexity Analysis

Time complexity:
O(b × m x s), where:

b = branching factor (number of possible domino placements).

m = maximum depth of Minimax exploration.

s = number of Monte Carlo simulations per move.

Space complexity:
O(b × m), dominated by storing cloned states per simulation depth.

This analysis indicates our Monte Carlo-enhanced approach has manageable computational complexity suitable for Domino's relatively small search space.

# Conclusion

In this project, we successfully combined Minimax and Monte Carlo simulations to create a robust AI player for Domino. By explicitly addressing the uncertainty inherent in hidden information games, our Monte Carlo enhanced Minimax algorithm significantly improved decision-making capabilities compared to traditional deterministic Minimax. Experimental results validate our method's effectiveness, outperforming human intuition in many cases. This combination of stochastic simulations with adversarial search represents a balanced approach, achieving strong performance without excessive computational demands. Our findings demonstrate the versatility of Monte Carlo methods in enhancing classical algorithms like Minimax, making them particularly effective for games involving uncertainty and partial observability.

# References

[1] S. Russell, P. Norvig, “Artificial Intelligence”, 3st ed., Pearson, Ed. Pearson, 2010.​

[2] GitHub Repository:

- UC Berkeley code repository, “aimacode” https://github.com/aimacode